This ipynb performs the last dea of the pipeline
- comparing each mono-culture sample to the co-culture
- comparing each cluster of the mono-co-culture combination against each other
- plots a volcano plot of the sample-to-sample dea

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import scanpy as sc
import gseapy
import matplotlib.pyplot as plt
import os
from bioinfokit import analys, visuz

In [ ]:
# astroco
path_to_adata = 'C:/Users/julia/Project/CCA_implementation_raw/Cell_type_annotation/BL_A_BL_C/AnnData_storage/BL_A_BL_C_annotated.h5ad'
astroco = sc.read_h5ad(os.path.join(path_to_adata))
# neuroco
path_to_adata = 'C:/Users/julia/Project/CCA_implementation_raw/Cell_type_annotation/BL_N_BL_C/AnnData_storage/BL_N_BL_C_annotated.h5ad'
neuroco = sc.read_h5ad(os.path.join(path_to_adata))

In [ ]:
astroco

In [ ]:
# astroco = astroco.raw.to_adata()  # solve accessing raw counts with maurits first, for now just use X of astroco
# neuroco = neuroco.raw.to_adata()

Perform DEA with mono-culture against co-culture for both samples

In [ ]:
sc.tl.rank_genes_groups(astroco, 
                        groupby='Sample', 
                        groups=['BL_C'],
                        method='wilcoxon',
                        corr_method='bonferroni',
                        pts=True)
# sc.tl.rank_genes_groups(neuroco, 
#                         groupby='Sample', 
#                         groups=['BL_C'],
#                         method='wilcoxon',
#                         corr_method='bonferroni',
#                         pts=True)

In [ ]:
# store results in tsv, for now take ALL p-value scores just like Maurits
BLCAdf = sc.get.rank_genes_groups_df(astroco, group='BL_C', key='rank_genes_groups', pval_cutoff=None, log2fc_min=None, log2fc_max=None, gene_symbols=None)
# BLCNdf = sc.get.rank_genes_groups_df(neuroco, group='BL_C', key='rank_genes_groups', pval_cutoff=None, log2fc_min=None, log2fc_max=None, gene_symbols=None)
BLCAdf.to_csv(os.path.join('C:/Users/julia/Project/Last_pipeline_scripts/GSEA', 'BLCA_rank_genes_groups.tsv'), sep='\t', encoding='utf-8')  
# BLCNdf.to_csv(os.path.join('C:/Users/julia/Project/big_dea_output/', 'BLCN_rank_genes_groups.tsv'), sep='\t', encoding='utf-8')  

Perform DEA in of the samples per cluster
- skip for now

Volcano plot for both samples:

In [ ]:
# I re-ran DEA for mono-culture against co-culture just to be sure
visuz.GeneExpression.volcano(df=BLCAdf, lfc="logfoldchanges", pv="pvals_adj", plotlegend=True, legendpos='upper right',
    color=('red', 'black', 'blue'), geneid="names", legendanchor=(1.46,1),figname='Volcano plot Astroco',
    gstyle=2, sign_line=True, xlm=(-50,50,5), ylm=(-50, 310,50), figtype='svg', axtickfontsize=10,
    lfc_thr=(1, 1), pv_thr=(0.05, 0.05), show=True)
# visuz.GeneExpression.volcano(df=BLCNdf, lfc="logfoldchanges", pv="pvals_adj", plotlegend=True, legendpos='upper right',
#     color=('red', 'black', 'blue'), geneid="names", legendanchor=(1.46,1), figname='Volcano plot Neuroco',
#     gstyle=2, sign_line=True, xlm=(-50,50,5), ylm=(-50, 310,50), figtype='svg', axtickfontsize=10,
#     lfc_thr=(1, 1), pv_thr=(0.05, 0.05), show=True)

--------------------------------------------------------------------------------------------------------

### Gene Set Enrichment Analyses (GSEA) with Scanpy ###

In [ ]:
adata = astroco.copy()

In [ ]:
adata.obs['seurat_annotated'] = adata.obs['kriegstein.seurat.custom.clusters.mean']
sc.pl.umap(adata, color='seurat_annotated', legend_loc='on data')

In [ ]:
from gseapy import Msigdb

In [ ]:
msig = Msigdb()

In [ ]:
categories = msig.list_category()

In [ ]:
len(categories)

In [ ]:
collection_counts = {}

for item in categories:
    parts = item.split('.')
    
    if len(parts) >= 1:
        collection = parts[0]
        if collection not in collection_counts:
            collection_counts[collection] = [item]
        else:
            collection_counts[collection].append(item)

print(collection_counts)

In [ ]:
usable_categories = []
collection_2_counts = {}
for key, value in collection_counts.items():
    if len(value) == 1:
        usable_categories.append(f'{value[0]}')
    else:
        filtered_values = [item for item in value if item.split('.')[1] != 'all']
        for i in filtered_values:
            parts =  i.split('.')
            second_category = parts[1]
            if second_category not in collection_2_counts:
                collection_2_counts[second_category] = [i]
            else:
                collection_2_counts[second_category].append(i)
for key, value in collection_2_counts.items():
    if len(value) == 1:
        usable_categories.append(f'{value[0]}')
    else:
        for i in value:
            parts = i.split('.')
            if len(parts) > 2:
                usable_categories.append(i)

print(usable_categories)

In [ ]:
usable_categories = (sorted(usable_categories))
print(usable_categories)

In [ ]:
# access gene list
glist = sc.get.rank_genes_groups_df(adata, group='BL_C', 
                                    key='rank_genes_groups', log2fc_min=0.25, 
                                    pval_cutoff=0.05)['names'].squeeze().str.strip().tolist()
print(len(glist))

He gives an error on 'c2.cp.kegg' --> what is that?

In [ ]:
usable_categories.remove('c2.cp.kegg') # remove this cause of strange bug?
usable_categories.remove('msigdb') # remove this because this is the entire database --> not necessery

In [ ]:
enrichr_list = []
gmt_list = []
for i in usable_categories:
    gmt = msig.get_gmt(category=i, dbver='2023.2.Hs')
    enr_res = gseapy.enrichr(gene_list=glist,
                     organism='Human',
                     gene_sets=gmt,
                     cutoff = 1)
    # change cutoff to 1 
    gmt_list.append(gmt)
    enrichr_list.append(enr_res)

In [ ]:
print(enrichr_list)

In [ ]:
gene_rank = sc.get.rank_genes_groups_df(adata, group='BL_C', key='rank_genes_groups')[['names','logfoldchanges']]
gene_rank.sort_values(by=['logfoldchanges'], inplace=True, ascending=False)

In [ ]:
gene_rank

In [ ]:
# calculate qc metrics to then filter out genes that occur in less then 30 cells
# (according to vignette, this is)
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
gene_rank = gene_rank[gene_rank['names'].isin(adata.var_names[adata.var.n_cells_by_counts>30])]
gene_rank

In [ ]:
for gmt, category_name in zip(gmt_list, usable_categories):
    res = gseapy.prerank(rnk=gene_rank, gene_sets=gmt)
    df = res.res2d
    df.to_csv(os.path.join('C:/Users/julia/Project/Last_pipeline_scripts/GSEA/', f'{category_name}.csv'))

In [ ]:
c5godb = pd.read_csv(os.path.join('C:/Users/julia/Project/Last_pipeline_scripts/GSEA/', 'c5.go.bp.csv'))

In [ ]:
c5godb

--------------------------------------------------------------------------------------------

In [ ]:
gmt = msig.get_gmt(category='c5.go.bp', dbver='2023.2.Hs')

In [ ]:
res = gseapy.prerank(rnk=gene_rank, gene_sets=gmt)

In [ ]:
res.res2d.head(10)

In [ ]:
res.ranking.shape # raking metric

In [ ]:
term = res.res2d.Term
# gp.gseaplot(res.ranking, term=term[i], **res.results[term[i]])
axs = res.plot(terms=term[:5])

In [ ]:
sc.tl.rank_genes_groups(adata, 
                        groupby='Sample', 
                        groups=['BL_C'],
                        method='wilcoxon',
                        corr_method='bonferroni',
                        pts=True)

In [ ]:
# get deg result
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
degs = pd.DataFrame(
    {group + '_' + key: result[key][group]
    for group in groups for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

In [ ]:
degs

In [ ]:
# subset up or down regulated genes
degs_sig = degs[degs.BL_C_pvals_adj < 0.05]
degs_up = degs_sig[degs_sig.BL_C_logfoldchanges > 0]
degs_dw = degs_sig[degs_sig.BL_C_logfoldchanges < 0]

In [ ]:
degs_up.shape

In [ ]:
degs_dw.shape

In [ ]:
gmt

In [ ]:
# Enricr API
enr_up = gseapy.enrichr(degs_up.BL_C_names,
                    gene_sets=gmt,
                    outdir=None)

In [ ]:
enr_up.res2d.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]

In [ ]:
gseapy.dotplot(enr_up.res2d, figsize=(3,5), title="Up", cmap = plt.cm.autumn_r)
plt.show()

In [ ]:
enr_dw = gseapy.enrichr(degs_dw.BL_C_names,
                    gene_sets=gmt,
                    outdir=None)

In [ ]:
enr_dw.res2d.Term = enr_dw.res2d.Term.str.split(" \(GO").str[0]
gseapy.dotplot(enr_dw.res2d,
           figsize=(3,5),
           title="Down",
           cmap = plt.cm.winter_r,
           size=5)
plt.show()

In [ ]:
# concat results
enr_up.res2d['UP_DW'] = "UP"
enr_dw.res2d['UP_DW'] = "DOWN"
enr_res = pd.concat([enr_up.res2d.head(), enr_dw.res2d.head()])
from gseapy.scipalette import SciPalette
sci = SciPalette()
NbDr = sci.create_colormap()
# display multi-datasets
ax = gseapy.dotplot(enr_res,figsize=(3,5),
                x='UP_DW',
                x_order = ["UP","DOWN"],
                title="GO_BP",
                cmap = NbDr.reversed(),
                size=3,
                cutoff=1,
                show_ring=True)
ax.set_xlabel("")
plt.show()

In [ ]:
enr_res

In [ ]:
ax = gseapy.barplot(enr_res, figsize=(3,5),
                group ='UP_DW',
                title ="GO_BP",
                cutoff=1,
                color = ['b','r'])

In [ ]:
import networkx as nx

In [ ]:
res.res2d.head()

In [ ]:
nodes, edges = gseapy.enrichment_map(res.res2d)

In [ ]:
nodes.head()

In [ ]:
edges.head()

In [ ]:
# build graph
G = nx.from_pandas_edgelist(edges,
                            source='src_idx',
                            target='targ_idx',
                            edge_attr=['jaccard_coef', 'overlap_coef', 'overlap_genes'])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# init node cooridnates
pos=nx.layout.spiral_layout(G)
#node_size = nx.get_node_attributes()
# draw node
nx.draw_networkx_nodes(G,
                       pos=pos,
                       cmap=plt.cm.RdYlBu,
                       node_color=list(nodes.NES),
                       node_size=list(nodes.Hits_ratio *1000))
# draw node label
nx.draw_networkx_labels(G,
                        pos=pos,
                        labels=nodes.Term.to_dict())
# draw edge
edge_weight = nx.get_edge_attributes(G, 'jaccard_coef').values()
nx.draw_networkx_edges(G,
                       pos=pos,
                       width=list(map(lambda x: x*10, edge_weight)),
                       edge_color='#CDDBD4')
plt.show()